# Data Mining Lab 2
In this lab session we will focus on the use of Neural Word Embeddings 

## Table of Contents
1. Data preparation
2. Feature engineering
3. Model
4. Results evaluation
5. Other things you could try
6. Deep Learning
7. Word to Vector
8. Clustering
9. High-dimension Visualization



### Necessary Library Requirements:

#### Same as Lab1:
- [Jupyter](http://jupyter.org/) (Strongly recommended but not required)
    - Install via `pip3 install jupyter` and use `jupyter notebook` in terminal to run
- [Scikit Learn](http://scikit-learn.org/stable/index.html)
    - Install via `pip3 sklearn` from a terminal
- [Pandas](http://pandas.pydata.org/)
    - Install via `pip3 install pandas` from a terminal
- [Numpy](http://www.numpy.org/)
    - Install via `pip3 install numpy` from a terminal
- [Matplotlib](https://matplotlib.org/)
    - Install via `pip3 maplotlib` from a terminal
- [Plotly](https://plot.ly/)
    - Install via `pip3 install plotly` from a terminal
- [Seaborn](https://seaborn.pydata.org/)
    - Install and signup for `seaborn`
- [NLTK](http://www.nltk.org/)
    - Install via `pip3 install nltk` from a terminal
    
#### New Libraries to intsall:
- [Gensim](https://pypi.org/project/gensim/)
    - Install via `pip3 install gensim`

- [Keras](https://keras.io/)
    - Install via `pip3 install keras`
    
                                                                                            
                                                                                           

---
## Introduction

**Dataset:** [SemEval 2017 Task](https://competitions.codalab.org/competitions/16380)

**Task:** Classify text data into 4 different emotions using word embedding and other deep information retrieval approaches.

![pic0](pics/pic0.png)

## 1. Data Preparation

Before beggining the lab, please make sure to download the [Google News Dataset](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit) and place it in a folder named "GoogleNews" in the same directory as this file.

# 1.1 Load data

We start by loading the csv files into a single pandas dataframe for training and one for testing.

In [229]:
conda install numpy


CondaHTTPError: HTTP 000 CONNECTION FAILED for url <https://repo.anaconda.com/pkgs/main/osx-64/current_repodata.json>
Elapsed: -

An HTTP error occurred when trying to retrieve this URL.
HTTP errors are often intermittent, and a simple retry will get you on your way.

If your current network has https://www.anaconda.com blocked, please file
a support request with your network engineering team.

'https://repo.anaconda.com/pkgs/main/osx-64'



Note: you may need to restart the kernel to use updated packages.


In [230]:
import pandas as pd

### training data

data_identification_df = pd.read_csv("data/data_identification.csv")
data_identification_df


,tweet_id,identification
0,0x28cc61,test
1,0x29e452,train
2,0x2b3819,train
3,0x2db41f,test
4,0x2a2acc,train
...,...,...
1867530,0x227e25,train
1867531,0x293813,train
1867532,0x1e1a7e,train
1867533,0x2156a5,train


In [231]:
emotions_df = pd.read_csv("data/emotion.csv")
emotions_df

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation
...,...,...
1455558,0x38dba0,joy
1455559,0x300ea2,joy
1455560,0x360b99,fear
1455561,0x22eecf,joy


In [232]:
tweets_df = pd.read_json('data/tweets_DM.json',lines=True)

# 1.2 joining 3 data set

In [233]:
tweets_df.iloc[5000]['_source']

{'tweet': {'hashtags': ['beer', 'benjaminfranklin'],
  'tweet_id': '0x2ff708',
  'text': '"Beer is proof that God loves us and wants us to be happy" - Benjamin Franklin  #beer <LH> #benjaminfranklin'}}

In [234]:
#Testing:  (to get tweets_id)

# The only key of the large dictionary of every row in _source is 'tweet'. From here we get a subdictionary, with keys
# hashtags, tweet_id, and text:

one_row = tweets_df.iloc[5000]['_source']  #grab only 1 row to test
new_dict = one_row['tweet']  

#Use the subkey found, tweet_id, to get the feature needed:
print(new_dict['tweet_id'])

#---------------------------------------------------------------

#def to get tweet_id from dictionary within a dictionary:
def get_tweet_id(row):
    return row['tweet']['tweet_id']

#def to get text from dictionary within a dictionary:
def get_text(row):
    return row['tweet']['text']

0x2ff708


In [235]:
tweets_df['tweet_id'] = tweets_df._source.apply(lambda x: get_tweet_id(x))
tweets_df['text'] = tweets_df._source.apply(lambda x: get_text(x))
tweets_df

,_score,_index,_source,_crawldate,_type,tweet_id,text
0,391,hashtag_tweets,"{'tweet': {'hashtags': ['Snapchat'], 'tweet_id...",2015-05-23 11:42:47,tweets,0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,hashtag_tweets,"{'tweet': {'hashtags': ['freepress', 'TrumpLeg...",2016-01-28 04:52:09,tweets,0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,hashtag_tweets,"{'tweet': {'hashtags': ['bibleverse'], 'tweet_...",2017-12-25 04:39:20,tweets,0x28b412,"Confident of your obedience, I write to you, k..."
3,376,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x1cd5...",2016-01-24 23:53:05,tweets,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2de2...",2016-01-08 17:18:59,tweets,0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...,...,...,...,...
1867530,827,hashtag_tweets,"{'tweet': {'hashtags': ['mixedfeeling', 'butim...",2015-05-12 12:51:52,tweets,0x316b80,When you buy the last 2 tickets remaining for ...
1867531,368,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x29d0...",2017-10-02 17:54:04,tweets,0x29d0cb,I swear all this hard work gone pay off one da...
1867532,498,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x2a6a...",2016-10-10 11:04:32,tweets,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,840,hashtag_tweets,"{'tweet': {'hashtags': [], 'tweet_id': '0x24fa...",2016-09-02 14:25:06,tweets,0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [236]:
twitter_data = tweets_df[["_score","tweet_id","text"]]

In [237]:
twitter_data

,_score,tweet_id,text
0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,0x28b412,"Confident of your obedience, I write to you, k..."
3,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...
1867530,827,0x316b80,When you buy the last 2 tickets remaining for ...
1867531,368,0x29d0cb,I swear all this hard work gone pay off one da...
1867532,498,0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,840,0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [238]:
twitter_data = twitter_data.merge(data_identification_df, how='inner')

In [239]:
print(len(twitter_data), twitter_data.tweet_id.nunique())

1867535 1867535


In [240]:
twitter_data.identification.unique()

array(['train', 'test'], dtype=object)

In [241]:
twitter_data.head()

,_score,tweet_id,text,identification
0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
2,232,0x28b412,"Confident of your obedience, I write to you, k...",test
3,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
4,989,0x2de201,"""Trust is not the same as faith. A friend is s...",test


# divide data to test and train

In [242]:
twitter_train = twitter_data[twitter_data['identification']== "train"].reset_index()
twitter_train

,index,_score,tweet_id,text,identification
0,0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train
1,1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train
2,3,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train
3,5,120,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train
4,6,1021,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train
...,...,...,...,...,...
1455558,1867526,94,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train
1455559,1867527,627,0x38959e,In every circumtance I'd like to be thankful t...,train
1455560,1867528,274,0x2cbca6,there's currently two girls walking around the...,train
1455561,1867533,840,0x24faed,"Ah, corporate life, where you can date <LH> us...",train


In [243]:
twitter_test = twitter_data[twitter_data['identification']== "test"].reset_index()
twitter_test = twitter_test.drop("index", axis = 1)
twitter_test

,_score,tweet_id,text,identification
0,232,0x28b412,"Confident of your obedience, I write to you, k...",test
1,989,0x2de201,"""Trust is not the same as faith. A friend is s...",test
2,66,0x218443,When do you have enough ? When are you satisfi...,test
3,104,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test
4,310,0x26289a,"In these tough times, who do YOU turn to as yo...",test
...,...,...,...,...
411967,602,0x2913b4,"""For this is the message that ye heard from th...",test
411968,598,0x2a980e,"""There is a lad here, which hath five barley l...",test
411969,827,0x316b80,When you buy the last 2 tickets remaining for ...,test
411970,368,0x29d0cb,I swear all this hard work gone pay off one da...,test


In [244]:
final_twitter_df = twitter_train.merge(emotions_df, how="inner")
final_twitter_df = final_twitter_df.drop("index",axis = 1) 

In [245]:
final_twitter_df

,_score,tweet_id,text,identification,emotion
0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation
1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness
2,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear
3,120,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy
4,1021,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation
...,...,...,...,...,...
1455558,94,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train,joy
1455559,627,0x38959e,In every circumtance I'd like to be thankful t...,train,joy
1455560,274,0x2cbca6,there's currently two girls walking around the...,train,joy
1455561,840,0x24faed,"Ah, corporate life, where you can date <LH> us...",train,joy


In [246]:
final_twitter_df.emotion.unique()

array(['anticipation', 'sadness', 'fear', 'joy', 'anger', 'trust',
       'disgust', 'surprise'], dtype=object)

# clean data

In [247]:
import re
import string

In [248]:
def clean(text):
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\[.*?\]', '', text)
     
    return text

In [249]:
final_twitter_df['text_clean'] = final_twitter_df.text.apply(clean)
final_twitter_df.iloc[0].text

'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'

In [250]:
final_twitter_df.head()

,_score,tweet_id,text,identification,emotion,text_clean
0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation,people who post add me on snapchat must be deh...
1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness,brianklaas as we see trump is dangerous to fre...
2,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear,now issa is stalking tasha 😂😂😂 lh
3,120,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy,riskshow thekevinallison thx for the best time...
4,1021,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation,still waiting on those supplies liscus lh


In [251]:
final_twitter_df

,_score,tweet_id,text,identification,emotion,text_clean
0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation,people who post add me on snapchat must be deh...
1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness,brianklaas as we see trump is dangerous to fre...
2,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear,now issa is stalking tasha 😂😂😂 lh
3,120,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy,riskshow thekevinallison thx for the best time...
4,1021,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation,still waiting on those supplies liscus lh
...,...,...,...,...,...,...
1455558,94,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train,joy,im so happy nowonder the name of this show hap...
1455559,627,0x38959e,In every circumtance I'd like to be thankful t...,train,joy,in every circumtance id like to be thankful to...
1455560,274,0x2cbca6,there's currently two girls walking around the...,train,joy,theres currently two girls walking around the ...
1455561,840,0x24faed,"Ah, corporate life, where you can date <LH> us...",train,joy,ah corporate life where you can date lh using ...


# save to csv

In [252]:
final_twitter_df.to_csv("predata.csv",index=False)

In [253]:
twitter_test.to_csv("twittertest_data.csv",index=False)

# feature

In [254]:
import pandas as pd
import numpy as np

In [255]:
twt = pd.read_csv('predata.csv', error_bad_lines=False, engine ='python')

In [256]:
pd.set_option('display.max_rows', 1000)
twt.head(1000)

,_score,tweet_id,text,identification,emotion,text_clean
0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation,people who post add me on snapchat must be deh...
1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness,brianklaas as we see trump is dangerous to fre...
2,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear,now issa is stalking tasha 😂😂😂 lh
3,120,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy,riskshow thekevinallison thx for the best time...
4,1021,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation,still waiting on those supplies liscus lh
5,481,0x368e95,Love knows no gender. 😢😭 <LH>,train,joy,love knows no gender 😢😭 lh
6,827,0x249c0c,@DStvNgCare @DStvNg More highlights are being ...,train,sadness,dstvngcare dstvng more highlights are being sh...
7,631,0x359db9,The #SSM debate; <LH> (a manufactured fantasy ...,train,anticipation,the ssm debate lh a manufactured fantasy used ...
8,839,0x23b037,I love suffering 🙃🙃 I love when valium does no...,train,joy,i love suffering 🙃🙃 i love when valium does no...
9,560,0x1fde89,Can someone tell my why my feeds scroll back t...,train,anger,can someone tell my why my feeds scroll back t...


In [257]:
twt.text_clean.isnull().sum()

5092

In [258]:
twt1 = twt.drop(twt[twt.text_clean.isnull()].index)

In [259]:
twt.iloc[200].text

'@Eyes_of_justice Well only <LH> off 1/2 doing something right'

In [260]:
twt1[twt1.text_clean.isnull()]

,_score,tweet_id,text,identification,emotion,text_clean


In [261]:
twt1

,_score,tweet_id,text,identification,emotion,text_clean
0,391,0x376b20,"People who post ""add me on #Snapchat"" must be ...",train,anticipation,people who post add me on snapchat must be deh...
1,433,0x2d5350,"@brianklaas As we see, Trump is dangerous to #...",train,sadness,brianklaas as we see trump is dangerous to fre...
2,376,0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>,train,fear,now issa is stalking tasha 😂😂😂 lh
3,120,0x1d755c,@RISKshow @TheKevinAllison Thx for the BEST TI...,train,joy,riskshow thekevinallison thx for the best time...
4,1021,0x2c91a8,Still waiting on those supplies Liscus. <LH>,train,anticipation,still waiting on those supplies liscus lh
...,...,...,...,...,...,...
1459411,94,0x321566,I'm SO HAPPY!!! #NoWonder the name of this sho...,train,joy,im so happy nowonder the name of this show hap...
1459412,627,0x38959e,In every circumtance I'd like to be thankful t...,train,joy,in every circumtance id like to be thankful to...
1459413,274,0x2cbca6,there's currently two girls walking around the...,train,joy,theres currently two girls walking around the ...
1459414,840,0x24faed,"Ah, corporate life, where you can date <LH> us...",train,joy,ah corporate life where you can date lh using ...


In [262]:

import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [263]:
BOW = CountVectorizer(tokenizer=nltk.word_tokenize, max_features=5000) #max_features=500

BOW.fit(twt1['text_clean'])

/Users/varidataweewattanapong/opt/anaconda3/envs/env/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(max_features=5000,
                tokenizer=<function word_tokenize at 0x7f7f80f271e0>)

In [264]:
feature_names = BOW.get_feature_names()

In [265]:
# 1. Learn a vocabulary dictionary of all tokens in the raw documents.
BOW_vectorizer.fit(twt1['text_clean'])

# 2. Transform documents to document-term matrix.
train_data_BOW_features = BOW_vectorizer.transform(twt1['text_clean'])
test_data_BOW_features = BOW_vectorizer.transform(twt1['text_clean'])

In [266]:
# check the result
train_data_BOW_features

<1454324x773670 sparse matrix of type '<class 'numpy.int64'>'
	with 18194212 stored elements in Compressed Sparse Row format>

In [267]:
# check the dimension
train_data_BOW_features.shape# observe some feature names


(1454324, 773670)

In [268]:
# observe some feature names
feature_names = BOW_vectorizer.get_feature_names()
feature_names[100:110]

['aaaaaw',
 'aaaaawww',
 'aaaaayyyyyye',
 'aaaab',
 'aaaadhiripoyindhiiii',
 'aaaadjdhdhsb',
 'aaaagain',
 'aaaages',
 'aaaagggghh',
 'aaaaggghhh']

In [269]:
"😂" in feature_names

False

In [270]:
import nltk

# build analyzers (bag-of-words)
BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(twt1['text_clean'])

train_data_BOW_features_500 = BOW_500.transform(twt1['text'])

## check dimension
train_data_BOW_features_500.shape

(1454324, 500)

In [271]:
train_data_BOW_features_500.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [272]:
# observe some feature names
feature_names_500 = BOW_500.get_feature_names()
feature_names_500[100:110]

['enjoy',
 'enough',
 'episode',
 'eurusd',
 'even',
 'ever',
 'every',
 'everyday',
 'everyone',
 'everything']

In [273]:
"😂" in feature_names_500

True

# model

In [274]:
from sklearn.tree import DecisionTreeClassifier

# for a classificaiton problem, you need to provide both training & testing data
X_train = BOW_500.transform(twt1['text_clean'])
y_train = twt1['emotion']

X_test = BOW_500.transform(twt1['text_clean'])
y_test = twt1['emotion']

## take a look at data dimension is a good habbit  :)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (1454324, 500)
y_train.shape:  (1454324,)
X_test.shape:  (1454324, 500)
y_test.shape:  (1454324,)


In [275]:
## build DecisionTree model
DT_model = DecisionTreeClassifier(random_state=0)

## training!
DT_model = DT_model.fit(X_train, y_train)

## predict!
y_train_pred = DT_model.predict(X_train)
y_test_pred = DT_model.predict(X_test)

## so we get the pred result
y_test_pred[:10]

array(['anticipation', 'sadness', 'disgust', 'joy', 'anticipation', 'joy',
       'sadness', 'anticipation', 'joy', 'anger'], dtype=object)

# thisss now

In [276]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    #return keras.utils.to_categorical(enc)
    return keras.utils.np_utils.to_categorical(enc)   #Allison/Moo said so, because of version

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 0    anticipation
1         sadness
2            fear
3             joy
Name: emotion, dtype: object

y_train.shape:  (1454324,)
y_test.shape:  (1454324,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]

y_train.shape:  (1454324, 8)
y_test.shape:  (1454324, 8)


In [277]:
X_train)

scipy.sparse.csr.csr_matrix

In [278]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [279]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 10000
X = model_input

# 1st hidden layer
X_W1 = Dense(units=32)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=32)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 500)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                16032     
_________________________________________________________________
re_lu_5 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_6 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 8)                 264       
_________________________________________________________________
softmax_3 (Softmax)          (None, 8)                 0   

In [280]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('training_log.csv')

# training setting
epochs = 10
batch_size = 20

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    validation_data = (X_test, y_test))
print('training finish')

Train on 1454324 samples, validate on 1454324 samples
Epoch 1/10
1454324/1454324 [==============================] - 352s 242us/step - loss: 1.4893 - accuracy: 0.4543 - val_loss: 1.4598 - val_accuracy: 0.4647
Epoch 2/10
1454324/1454324 [==============================] - 349s 240us/step - loss: 1.4583 - accuracy: 0.4661 - val_loss: 1.4479 - val_accuracy: 0.4695
Epoch 3/10
1454324/1454324 [==============================] - 355s 244us/step - loss: 1.4515 - accuracy: 0.4685 - val_loss: 1.4442 - val_accuracy: 0.4714
Epoch 4/10
1454324/1454324 [==============================] - 348s 239us/step - loss: 1.4482 - accuracy: 0.4699 - val_loss: 1.4419 - val_accuracy: 0.4721
Epoch 5/10
1454324/1454324 [==============================] - 348s 239us/step - loss: 1.4461 - accuracy: 0.4710 - val_loss: 1.4386 - val_accuracy: 0.4738
Epoch 6/10
1454324/1454324 [==============================] - 349s 240us/step - loss: 1.4446 - accuracy: 0.4714 - val_loss: 1.4402 - val_accuracy: 0.4730
Epoch 7/10
1454324/145

In [293]:
X_test = BOW_500.fit_transform(twitter_test['text'])

# result evaluation

In [298]:
X_train

<411972x500 sparse matrix of type '<class 'numpy.int64'>'
	with 6333595 stored elements in Compressed Sparse Row format>

In [299]:
## predict
pred_result = model.predict(X_test)
pred_result[:5]

array([[0.00761052, 0.19282967, 0.01605352, 0.00538574, 0.49761197,
        0.1265756 , 0.01034839, 0.1435847 ],
       [0.00691766, 0.27195   , 0.01407202, 0.00358874, 0.44054574,
        0.12858893, 0.00644224, 0.12789476],
       [0.02198055, 0.02032033, 0.43286777, 0.02048348, 0.08704505,
        0.34802437, 0.03254484, 0.03673367],
       [0.0009173 , 0.74409956, 0.00288838, 0.00086896, 0.13993977,
        0.01239417, 0.00603875, 0.09285303],
       [0.02485532, 0.45214266, 0.04858831, 0.0109105 , 0.11642832,
        0.20435502, 0.02133618, 0.12138376]], dtype=float32)

In [300]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['joy', 'joy', 'disgust', 'anticipation', 'anticipation'],
      dtype=object)

In [301]:
twitter_test

,_score,tweet_id,text,identification
0,232,0x28b412,"Confident of your obedience, I write to you, k...",test
1,989,0x2de201,"""Trust is not the same as faith. A friend is s...",test
2,66,0x218443,When do you have enough ? When are you satisfi...,test
3,104,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test
4,310,0x26289a,"In these tough times, who do YOU turn to as yo...",test
...,...,...,...,...
411967,602,0x2913b4,"""For this is the message that ye heard from th...",test
411968,598,0x2a980e,"""There is a lad here, which hath five barley l...",test
411969,827,0x316b80,When you buy the last 2 tickets remaining for ...,test
411970,368,0x29d0cb,I swear all this hard work gone pay off one da...,test


In [302]:
twitter_test['emotion']=pred_result

In [303]:
twitter_test

,_score,tweet_id,text,identification,emotion
0,232,0x28b412,"Confident of your obedience, I write to you, k...",test,joy
1,989,0x2de201,"""Trust is not the same as faith. A friend is s...",test,joy
2,66,0x218443,When do you have enough ? When are you satisfi...,test,disgust
3,104,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test,anticipation
4,310,0x26289a,"In these tough times, who do YOU turn to as yo...",test,anticipation
...,...,...,...,...,...
411967,602,0x2913b4,"""For this is the message that ye heard from th...",test,joy
411968,598,0x2a980e,"""There is a lad here, which hath five barley l...",test,joy
411969,827,0x316b80,When you buy the last 2 tickets remaining for ...,test,joy
411970,368,0x29d0cb,I swear all this hard work gone pay off one da...,test,joy


# export

In [305]:
twitter_test[['emotion','tweet_id']].to_csv("submit.csv")

Student Information

Name: Varida

Student ID: 110065421

GitHub ID: varidatawee

Kaggle name: T